In [5]:
import pandas as pd
import xlrd
import os
import warnings

file_paths = ["Rocznik_2014__GR.xls", "Rocznik_2015__GR.xls", "Rocznik_2016__GR.xls", "Rocznik_2017_GR.xls", "Rocznik_2018_GR.xls", "Rocznik_2019_GR.xls",
              "Rocznik_2020_GR.xls", "Rocznik_2021_GR.xls", "Rocznik_2022_GR.xls"]

# map files to tab names [tabs are mixed between years]
files_to_tab_names = dict()
headers = dict()
for ind, path in enumerate(file_paths):
  files_to_tab_names[path] = dict()
  headers[path] = dict()

  if ind <= 3:
    files_to_tab_names[path]["market_value"] = "Tab 17"
  else:
    files_to_tab_names[path]["market_value"] = "Tab 12"

In [31]:
def read_excel_files(path):
    excel_data = {}
    for file in os.listdir(path):
        if file.endswith(".xls"):
            year = file.split('_')[1]  # Extracting year from file name
            xls = pd.ExcelFile(os.path.join(path, file))
            for sheet_name in xls.sheet_names:
                if sheet_name == files_to_tab_names[file]["market_value"]:
                    df = pd.read_excel(xls, sheet_name=sheet_name, header=3)
                    for index, row in df.iterrows():
                      company = row[1]
                      if company not in excel_data:
                        excel_data[company] = {}
                      if int(year) >= 2018:
                        excel_data[company][year] = row[5]
                      else:
                        excel_data[company][year] = row[3]

    return excel_data

excel_data = read_excel_files("./")
excel_data = {key: dict(sorted(value.items(), reverse=True)) for key, value in excel_data.items()}
for company, data in excel_data.items():
  for year, value in data.items():
   if str(int(year) - 1) in excel_data[company].keys() and excel_data[company][str(int(year) - 1)] != 0:
      excel_data[company][year] = 100*(value - excel_data[company][str(int(year) - 1)])/excel_data[company][str(int(year) - 1)]

# Create DataFrame
df = pd.DataFrame.from_dict(excel_data, orient='index')
print(df)
df.to_csv('market_values.csv', index=True)

WARNING *** file size (1183124) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
dict_keys(['2020', '2016', '2022', '2018', '2015', '2014', '2021', '2017', '2019'])
